In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
import numpy as np

In [ ]:
check = ((Y_train == 1) | (Y_train == 0)) 
print(check.shape)
X_train = X_train[check]
Y_train = Y_train[check]
#[0 1 2] -> [1 0 0] [0 1 0] [0 0 1]
#vector zeros co do dai bang so class -> vector độ dài C -> [0, 0, 0..., 0]-> dữ liệu đc gán nhãn j -> y[j] = 1
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train)
print(Y_train.shape)

(60000,)
(12665, 2)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])/255.
X_train = np.hstack((X_train, np.zeros((X_train.shape[0], 1))))
print(X_train.shape)

(12665, 785)


In [ ]:
def softmax(Z):
  #print(Z.shape)
  #e^a/(e^a + e^b + e^c) = e^(a - c) / e^(a-c) + e^(b-c) + 1
  max_Z = np.max(Z, axis = 1)
  #Z co shape (N, classes)
  #print(max_Z.shape)
  Z_t = Z.T
  Z_t = Z_t - max_Z
  new_Z = Z_t.T
  new_Z = np.exp(new_Z)
  sum_eZT = np.sum(new_Z, axis = 1).reshape(-1)
  ez_T = new_Z.T
  ez_T /= sum_eZT
  ez = ez_T.T
  return ez
x = np.array([[1, 2, 3],
              [3, 4, 5],
              [4, 5, 6],
              ])
#[3, 5, 6]
#[[1, 3, 4], [2, 4, 5] [3, 5, 6]] - [3, 5, 6] = [[1, 3, 4]- [3, 5, 6], [2, 4, 5]- [3, 5, 6], [3, 5, 6] - [3, 5, 6]] 
print(np.sum(softmax(x),axis = 1))

[1. 1. 1.]


In [ ]:
def relu(Z):
  new_Z = Z
  new_Z[new_Z < 0] = 0.
  return new_Z
def sigmoid(Z):
  return 1/(np.exp(-Z) + 1)
def activation(Z, type):
  if type == "relu":
    return relu(Z)
  if type == "softmax":
    return softmax(Z)
  if type == "sigmoid":
    return sigmoid(Z)
  return Z
def gradient(Z, type):
  if type == "relu":
    new_Z = np.ones(Z.shape)
    new_Z[new_Z < 0] = 0.
    return new_Z
  if type == "softmax":
    return softmax(Z) * (1 - softmax(Z))
  if type == "sigmoid":
    return sigmoid(Z) * (1 -sigmoid(Z))
  return np.ones(Z.shape)
def crossentropy_loss(Z, Y):
  return -np.sum(np.sum(Y * np.log(Z), axis = 1),axis = 0)/Y.shape[0]
class NeuralNetwork():
  def __init__(self, layers_node, layers_activation, input, output):
    #(200, 10)-> 128 -> [200, 128]-> (10, 128)
    #layers_information : [(128, ""), (128, "softmax")]
    self.layers_activation = layers_activation
    self.layers_node = layers_node
    #np.random.rand(self.layers_node[index - 1], layer_node)
    #np.full((input.shape[1], layer_node), 0.1)
    #np.full((self.layers_node[index - 1], layer_node)
    self.weights = [(np.random.rand(input.shape[1], layer_node) if index == 0 else np.random.rand(self.layers_node[index - 1], layer_node)) 
                      for (index, layer_node) in enumerate(self.layers_node)]
    self.input = input
    self.output = output
  def feed_forward(self, input):
    output = input.copy()
    Z = []
    A = []
    A.append(input)
    for index, weight in enumerate(self.weights):
      output = output.dot(weight)
      Z.append(output)
      output = activation(output, self.layers_activation[index])
      A.append(output)
    return Z, A
  def backpropagation(self, input, output, lr):
      Z, A = self.feed_forward(input)
      #print(Z[0].shape)
      #print(A[0].shape)
      E = []
      for i in range(len(self.weights) - 1, -1, -1):
        if i == len(self.weights) - 1:
          E.append(A[i + 1] - output)
        else:
          E.append((E[-1].dot(self.weights[i + 1].T))* gradient(Z[i], self.layers_activation[i]))
        #print(A[i - 1].shape)
        #print(E[-1].shape)
        self.weights[i] -= lr * (A[i].T.dot(E[-1]))
      return Z, A
  def training(self, lr, number_iterations, batch_size):
    for epoch in range(number_iterations):
      random = np.random.permutation(int(X_train.shape[0]/batch_size))
      sum_loss = 0
      for j in random:
        Xj = self.input[j * batch_size : (j + 1) * batch_size]
        Yj = self.output[j * batch_size : (j + 1) * batch_size]
        Z, A = self.backpropagation(Xj, Yj, lr)
        sum_loss += crossentropy_loss(A[-1], Yj)
      if epoch % 10 == 0:
        print(sum_loss / random.shape[0])
model = NeuralNetwork([128, 128, 2], ["sigmoid", "sigmoid", "softmax"], X_train, Y_train)
model.training(0.00001, 1000, 32)
#4600 = N 
#200, 100
#4600 , 100
#100 , 20
#4600, 20 * 20, 100 = 4600, 100

379.6854150211065
273.0274957441584
272.9995319812653
273.00175839022353
272.9915426073169
273.03208028480105
273.0246217582235
273.02017126889353
273.03792874008394
273.0258544851149
273.0388533415231
273.0223426737837
273.0368668373593
273.03749909339786
273.00941789316875
273.04028324470596
273.0093179267153
273.0022328880753
273.00167179475477
273.0219932833992
273.0136057145978
273.044687938287
272.98353895425873
272.98592974357774
273.00867196763494
273.0280072310538
273.0137883043949
272.9223299864345
273.0251138403501
273.02192805219784
273.0065224219725
273.03220701805236
273.03297906207337
272.9936111695796
272.9918986407755
273.0556562469772
273.01478842892
273.0446936212702
273.0276201945138
273.02853995124946
273.0389412520207
273.01837978196096
272.9784949305759
273.02757671549773
273.0202313298879
273.0378533947048
272.983708404383
272.99336251631894
273.0231269937286
273.03469259804797
272.98659217071406
273.0127594455659
273.00827538809415
273.02656702354835
273.041413